# Fullstack GPT: #5.0 ~ #5.8

- [ ] Implement an LCEL chain with a memory that uses one of the memory classes we learned about.
The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
- [ ] Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.
- [ ] To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.